In [5]:
import re
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


### Read data from IBMS

In [2]:
#%%bigquery df_synopsys_BQ
#SELECT * FROM `gglobo-coe-di-gjcd-dev-dev-qa.ASL_project.synopsis_From_IBMS`

In [3]:
#df_synopsys_BQ.to_csv("gs://asl-project-de3c21f6/raw_data/synopsis_From_BQ.csv", index=False)

In [6]:
df_synopsys_raw = pd.read_csv("gs://asl-project-globo-de18f040/data/raw/synopsis_From_BQ.csv")
df_synopsys_raw.head()

,type,title,synopsis,genre
0,FILME,Brooklin's Finest,"Três policiais do Brooklyn - Eddie Dugan, Tango e Sal -, com carreiras distintas, ficam presos na mesma cena de um crime. Juntos, eles devem solucionar o caso, mas, para isso, serão testadas a lealdade e a moral.",Crime
1,FILME,Cogan's Trade (Aka: Killing Them Softly),"JACKIE COGAN É UM MATADOR PROFISSIONAL CONTRATADO PARA INVESTIGAR UM ASSALTO, OCORRIDO EM MEIO A UMA PARTIDA DE PÓQUER PROTEGIDA PELA MÁFIA. .",Crime
2,FILME,Inocência,"No Brasil imperial, um médico itinerante conhece uma moça com malária por quem se apaixona, sendo correspondido. Entretanto, o pai da jovem a prometeu para um rico fazendeiro.",Drama
3,FILME,"Beijo Da Mulher-Aranha, O","Nos porões da ditadura latino-americana, dois presos são obrigados a conviver na mesma cela. Molina é condenado por corrupção de menores; Valentin é torturado pelo governo para delatar companheiros.",Drama
4,FILME,Carandiru,"Um médico se oferece para realizar um trabalho de prevenção à AIDS no Carandiru. Lá ele convive com a realidade dos presos, como violência, superlotação e instalações precárias.",Drama


## Dataclean

In [10]:
def prep_genre(genre):
    
    output = list()
    
    for i in genre:
        
        genre_string = i.strip()
        
        if len(genre_string) > 0:
            output.append(genre_string)
        
    output_str = ' '.join([str(elem) for elem in output])
    
    output_uniques =  list(set(output_str.split(' ')))
    
    final_output = ' '.join([str(elem) for elem in output_uniques])

    return final_output

In [11]:
def replace_artigo(nome_programa):
    aconteceu = 0
    if nome_programa[-3:] == ", A":      
        padrao = ", A"
        sub = "A"
        aconteceu = 1
    elif nome_programa[-4:] == ", As":
        padrao = ", As"
        sub = "As"
        aconteceu = 1
    elif nome_programa[-3:] == ", O":      
        padrao = ", O"
        sub = "O"
        aconteceu = 1
    elif nome_programa[-4:] == ", Os":
        padrao = ", Os"
        sub = "Os"
        aconteceu = 1
    elif nome_programa[-5:] == ", The":
        padrao = ", The"
        sub = "The"
        aconteceu = 1
    
    if aconteceu == 1:
        temp_var = re.sub(padrao,"", nome_programa)
        return sub + " " + temp_var
    else:
        return nome_programa

In [12]:
df_synopsys_raw['title'] = df_synopsys_raw['title'].apply(lambda x: replace_artigo(x))
df_synopsys_raw['genre'] = df_synopsys_raw['genre'].str.replace("|", ",", regex=True)
df_synopsys_raw['genre'] = df_synopsys_raw['genre'].str.replace("/", ",", regex=True)
df_synopsys_raw['genre'] = df_synopsys_raw['genre'].apply(lambda x: x.split(','))
df_synopsys_raw['genre'] = df_synopsys_raw['genre'].apply(lambda x: prep_genre(x))
df_synopsys_raw['genre'] = df_synopsys_raw[['type', 'title', 'genre']].groupby(['type', 'title'])['genre'].transform(lambda x: ' '.join(x))
df_synopsys_raw['synopsis_count'] = df_synopsys_raw['synopsis'].apply(lambda x: len(x))
df_synopsys_raw['synopsis_max'] = df_synopsys_raw[['type', 'title', 'synopsis','synopsis_count']].groupby(['type', 'title'])['synopsis_count'].transform(max)
df_synopsys_raw = df_synopsys_raw[df_synopsys_raw.synopsis_count == df_synopsys_raw.synopsis_max][['type', 'title', 'synopsis', 'genre']]
df_synopsys_raw['genre'] = df_synopsys_raw['genre'].apply(lambda x: list(set(x.split(" "))))
df_synopsys_raw['genre'] = df_synopsys_raw['genre'].apply(lambda x: " ".join(x))

df_final = df_synopsys_raw[['type', 'title', 'synopsis','genre']].groupby(['type', 'title', 'synopsis','genre']).size().reset_index(name = "count_duplicates")

df_final = df_final[['type', 'title', 'synopsis', 'genre']]

df_final.shape

(1618, 4)

In [13]:
df_final.to_csv("gs://asl-project-globo-de18f040/data/raw/synopsis_clean.csv", index=False)